<a href="https://colab.research.google.com/github/hpt09/ResumeParser/blob/master/SpaCy_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy
!pip install PyMuPDF

In [ ]:
import spacy
import pickle
import random

In [ ]:
##### TRAIN NLP_MODEL #####

train_data = pickle.load(open('train_data.pkl','rb'))
print(f"Training data consist of {len(train_data)} manually labelled resume's.")
# loading blank spacy model as we want to customize our model.
# spacy.blank('en') will create a blank model of a given language class i.e., for here English.
nlp = spacy.blank('en')
# Creating a function to train our model
def train_model(train_data):
  
  if 'ner' not in nlp.pipe_names:# Checking if NER is present in pipeline
    ner = nlp.create_pipe('ner') # creating NER pipe if not present
    nlp.add_pipe(ner, last=True) # adding NER pipe in the end

  for _, annotation in train_data: # Getting 1 resume at a time from our training data of 200 resumes
    for ent in annotation['entities']: # Getting each tuple at a time from 'entities' key in dictionary at index[1] i.e.,(0, 15, 'Name') and so on
      ner.add_label(ent[2])  # here we are adding only labels of each tuple from entities key dict, eg:- 'Name' label of (0, 15, 'Name')
    
  # In above for loop we finally added all custom NER from training data.
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner'] # getting all other pipes except NER.
  with nlp.disable_pipes(*other_pipes): # Disabling other pipe's as we want to train only NER.
        optimizer = nlp.begin_training()
        
        for itn in range(10):         # trainig model for 10 iteraion
            print('Starting iteration ' + str(itn))
            random.shuffle(train_data) # shuffling data in every iteration 
            losses = {}
            for text, annotations in train_data:
              try:
                nlp.update(
                    [text],        #batch of texts
                    [annotations], #batch of annotations
                    drop=0.2,      #dropout rate -makes it harder to memorise
                    sgd=optimizer, #callable to update weights
                    losses=losses) #Dictionary to update with the loss, keyed by pipeline component.
              except Exception as e:
                pass


train_model(train_data)
# Saving our trained model to re-use.
nlp.to_disk('nlp_model')
# Loading our trained model
nlp_model = spacy.load('nlp_model')
# Checking all the custom NER created
nlp_model.entity.labels

In [ ]:
##### TEST NPL_MODEL WITH CVS IN CVS/ DIRECTORY

import os,sys,fitz
cvs = []
for file in os.listdir('cvs'):
  doc = fitz.open('cvs/'+file)
  cv = ""
  for page in doc:
    cv = cv + str(page.getText())
  
  cvs.append(cv)



r = cvs[2]
r = r.replace("●","")
r = r.replace("","")
r = r.replace("•","")
r = r.replace("▪","")


doc = nlp_model(" ".join(r.split('\n')))
for ent in doc.ents:
  print(f'{ent.label_.upper():{20}} - {ent.text}')

NAME                 - NATHAN BARNES
LOCATION             - Auckland
DEGREE               - Bachelor of Creative Software
SKILLS               - Computer literate: MS Word, Excel,  Publisher, Emails, Internet    Coding languages : C# , HTML,  Python, C++, Java, Javascript   Game Development          PERSONAL SKILLS    Communication  - ability to relate & communicate with all people     Customer Service Skills - ability to help and communicate with customers professionally     Team Player  - Able to work effectively with others    Overcome obstacles - If a problem arises, being able to take action and get the job done    INTERESTS   Developing   Fixing and/or solving a problem   Helping others   Interacting with others   Traveling   Game and App Design      REFEREES
DESIGNATION          - Operations Manager
